In [5]:
import sys
import os

# Go two levels up: from 'notebooks/task1/' to the project root
sys.path.append(os.path.abspath(os.path.join('..', '..')))

In [6]:
import pandas as pd
import glob
import os

# Path to yfinance_data folder
yfinance_folder = r"C:\Users\ABC\Desktop\10Acadamy\week1\analysis-of-financial-news-data\data\yfinance_data"

# Get all CSVs in folder
csv_files = glob.glob(os.path.join(yfinance_folder, "*_historical_data.csv"))

# Read and stack all yfinance CSVs
df_list = []
for file in csv_files:
    # Extract ticker from filename (e.g. AAPL from AAPL_historical_data.csv)
    ticker = os.path.basename(file).split('_')[0].upper()
    df = pd.read_csv(file)
    df['stock'] = ticker
    df_list.append(df)

df_prices = pd.concat(df_list, ignore_index=True)
# Use format='mixed' to handle different date formats
df_prices['date'] = pd.to_datetime(df_prices['Date'], format='mixed').dt.strftime('%Y-%m-%d')
df_prices['stock'] = df_prices['stock'].str.upper()

# (Optional) Keep only necessary columns for the merge
cols_keep = ['date', 'stock', 'Close', 'Open', 'High', 'Low', 'Volume']
df_prices = df_prices[cols_keep]

# Load news data
news_path = r"C:\Users\ABC\Desktop\10Acadamy\week1\analysis-of-financial-news-data\data\news_data_for_stock\clean_analyst_rating.csv"
df_news = pd.read_csv(news_path)
df_news['stock'] = df_news['stock'].replace({'FB': 'META', 'MSF': 'MSFT'})
df_news['date'] = pd.to_datetime(df_news['date'], format='mixed').dt.strftime('%Y-%m-%d')
df_news['stock'] = df_news['stock'].str.upper()

# Merge (inner join on 'date' and 'stock')
df_merged = pd.merge(
    df_news,
    df_prices,
    on=['stock', 'date'],
    how='inner'
)

# Save to CSV
out_path = r"C:\Users\ABC\Desktop\10Acadamy\week1\analysis-of-financial-news-data\data\news_yfinance_merged.csv"
df_merged.to_csv(out_path, index=False)

print(f"✅ Merged data saved: {out_path}")
print(df_merged.head())



✅ Merged data saved: C:\Users\ABC\Desktop\10Acadamy\week1\analysis-of-financial-news-data\data\news_yfinance_merged.csv
   Unnamed: 0                                           headline  \
0        7120  Tech Stocks And FAANGS Strong Again To Start D...   
1        7121      10 Biggest Price Target Changes For Wednesday   
2        7122  Benzinga Pro's Top 5 Stocks To Watch For Wed.,...   
3        7123  Deutsche Bank Maintains Buy on Apple, Raises P...   
4        7124  Apple To Let Users Trade In Their Mac Computer...   

                                                 url          publisher  \
0  https://www.benzinga.com/government/20/06/1622...         JJ Kinahan   
1  https://www.benzinga.com/analyst-ratings/price...         Lisa Levin   
2  https://www.benzinga.com/short-sellers/20/06/1...  Benzinga Newsdesk   
3  https://www.benzinga.com/news/20/06/16219873/d...  Benzinga Newsdesk   
4  https://www.benzinga.com/news/20/06/16218697/a...      Neer Varshney   

         date stock 